In [1]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string
import csv
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
import sklearn.metrics
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

In [2]:
def parseLabeledData(path):
    file=open(path, 'r')
    dataList = []
    for line in csv.reader(file):        
        if len(line) >= 5:
            dataList.append(
                {"asin":line[0], 
                 "question":line[1],
                 "review":line[2],
                 "answer":line[3],
                 "label":line[4]}
            )     
    return dataList
        

print("Reading labeled data...")
#data = parseLabeledData("C:/Users/Moi/Downloads/out.csv")
data = parseLabeledData("/Users/Silvia/Desktop/New Data - Sheet1.csv")
queries = [d['question'] for d in data]
answers = [d['answer'] for d in data]
reviews = [d['review'] for d in data]
labels = [d['label'] for d in data]
print("done")

Reading labeled data...
done


In [3]:
def parseAllQueries(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

def parseAllReviews(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

print("Reading all reviews & all questions...")

#allReviews = parseAllReviews("C:/Users/Moi/Downloads/reviews.json")
#allQuestions = parseAllQueries("C:/Users/Moi/Downloads/qa.json")

allReviews = parseAllReviews("/Users/Silvia/Downloads/reviews.json")
allQuestions = parseAllQueries("/Users/Silvia/Downloads/qa.json")

# do we have to remove questions that have no reviews or reviews that have no questions??
docSet = []
for entry in allReviews.values():
    for review in entry:
        docSet.append(review["reviewText"])

for entry in allQuestions.values():
    for question in entry:
        docSet.append(question["question"])

docLen = [len(d.split()) for d in docSet]
avgdl = sum(docLen) / len(docLen)

print("done")

Reading all reviews & all questions...
done


In [4]:
def countAllWords():
    allWords = defaultdict(int)
    englishStopWords = stopwords.words('english')
    for r in allReviews.values():
        for review in r:
            review = review["reviewText"]
            exclude = set(string.punctuation)
            review = ''.join(ch for ch in review if ch not in exclude)
            for w in review.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1

    for q in allQuestions.values():
        for question in q:
            question = question["question"]
            exclude = set(string.punctuation)
            question = ''.join(ch for ch in question if ch not in exclude)
            for w in question.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1
    
    
    return allWords

allWords = countAllWords()

In [14]:
commonWords = sorted(allWords, key=lambda x: -allWords[x])[:5000]

In [21]:
def wordToIndex(term):
    if term in commonWords:
        return commonWords.index(term)
    else:
        return -1

whos


In [27]:
def pairwiseProduct(review, question, length):
    reviewBag = [0]*length
    questionBag = [0]*length
    
    exclude = set(string.punctuation)
    review = ''.join(ch for ch in review if ch not in exclude)
    question = ''.join(ch for ch in question if ch not in exclude)
    
    for term in review.lower().split():
        index = wordToIndex(term)
        
        if index >= 0 and index < length:
            reviewBag[index] = review.lower().split().count(term)
        
    for term in question.lower().split():
        index = wordToIndex(term)
        
        if index >= 0 and index < length:
            questionBag[index] = question.lower().split().count(term)
        
    bagFeat = [0]*length
    
    for i in range(0, length):
        bagFeat[i] = reviewBag[i] * questionBag[i]
        
    #for i in range(0, length):
        #if reviewBag[i] > 0 or questionBag[i] > 0:
            #print(commonWords[i], reviewBag[i], questionBag[i], bagFeat[i])
        
    return bagFeat        

In [28]:
def theLavFun(review, question):
    feat = [1]
    feat.append(pairwiseProduct(review, question, 5000))
    return feat

In [29]:
def train(X, y): 
    lr = LinearRegression()
    lr.fit(X, y)
    
    return lr


X = [theLavFun(d["review"], d["question"]) for d in data]
y = [1 if a == "Y" else 0 for a in answers]

random.seed(4410)

for j in range(0,len(X[0])):
    featList = []
    for i in range(0,len(X)-1):
        featList.append(X[i][j])
    featList = normalize(featList)
    for i in range(0,len(X)-1):
        X[i][j] = featList[i]

keys = list(range(1, len(labels)))
points = dict(zip(keys, zip(X, y)))
random.shuffle(keys)
X_rand = [points[key][0] for key in keys]
y_rand = [points[key][1] for key in keys]

X_train = X[:len(X_rand)*2//3]
y_train = y[:len(y_rand)*2//3]

X_test = X[len(X_rand)*2//3:]
y_test = y[len(y_rand)*2//3:]

train(X_train, y_train)

y_hatTrain = lr.predict(X_train)
y_hatTest = lr.predict(X_test)

#we done

NameError: name 'normalize' is not defined